In [4]:
import json

In [9]:
import torch

In [14]:
import pandas as pd
import os
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers.optimization import  Adafactor 
import time
import warnings

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [3]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

/home/lg/PycharmProjects/venvs/my/samples/venv/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 242M/242M [00:21<00:00, 11.5MB/s]


In [7]:
with open('./text_generation/out_all.json') as file: 
    raw = file.read()
    data = json.loads(raw)
    
data[0]['title']

'Parimatch Casino India | Get 150% up to ₹105,000 for signing up'

In [43]:
if torch.cuda.is_available():
    dev = torch.device("cuda:0") 
    print("Running on the GPU")
else:
    dev = torch.device("cpu")
    print("Running on the CPU")
    
dev = torch.device("cpu")

Running on the GPU


In [44]:
model.to(dev)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [28]:
batch_size=1
num_of_batches=len(data)/batch_size
num_of_epochs=4
num_of_batches=int(num_of_batches)
num_of_batches

110

In [15]:
optimizer = Adafactor(
    model.parameters(),
    lr=1e-3,
    eps=(1e-30, 1e-3),
    clip_threshold=1.0,
    decay_rate=-0.8,
    beta1=None,
    weight_decay=0.0,
    relative_step=False,
    scale_parameter=False,
    warmup_init=False
)

In [16]:

from IPython.display import HTML, display

def progress(loss,value, max=100):
    return HTML(""" Batch loss :{loss}
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(loss=loss,value=value, max=max))

In [17]:
num_of_epochs=1

In [35]:
torch.cuda.empty_cache()

In [24]:
%env PYTORCH_NO_CUDA_MEMORY_CACHING=0

env: PYTORCH_NO_CUDA_MEMORY_CACHING=0


In [37]:
if torch.cuda.is_available():
    model.convert_to_fp16()

AttributeError: 'T5ForConditionalGeneration' object has no attribute 'convert_to_fp16'

In [ ]:
#Sets the module in training mode
model.train()
labelbatch
loss_per_10_steps=[]
for epoch in range(1,num_of_epochs+1):
    
    print('Running epoch: {}'.format(epoch))
  
    running_loss=0

    out = display(progress(1, num_of_batches+1), display_id=True)
    for i in range(num_of_batches):
        inputbatch=[]
        labelbatch=[]
        new_df=data[i*batch_size:i*batch_size+batch_size]
        for exmp in new_df:
            input = f'<title> {exmp["title"]} <description> {exmp["description"]} <keywords> {exmp["keywords"]}'+'</s>' 
            labels = exmp["text"]["text"]+'</s>' [:50]  
            inputbatch.append(input)
            labelbatch.append(labels)
    

        inputbatch=tokenizer.batch_encode_plus(inputbatch,padding=True,max_length=128,return_tensors='pt')["input_ids"]
        labelbatch=tokenizer.batch_encode_plus(labelbatch,padding=True,max_length=512,return_tensors="pt") ["input_ids"]
        inputbatch=inputbatch.to(dev)
        labelbatch=labelbatch.to(dev)

        # clear out the gradients of all Variables 
        optimizer.zero_grad()

        # Forward propogation
        outputs = model(input_ids=inputbatch, labels=labelbatch)
        loss = outputs.loss
        loss_num=loss.item()
        logits = outputs.logits
        running_loss+=loss_num
        if i%10 ==0:      
            loss_per_10_steps.append(loss_num)
        
        out.update(progress(loss_num,i, num_of_batches+1))

        # calculating the gradients
        loss.backward()

        #updating the params
        optimizer.step()

    running_loss=running_loss/int(num_of_batches)
    print('Epoch: {} , Running loss: {}'.format(epoch,running_loss))

Running epoch: 1


In [30]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 6            |        cudaMalloc retries: 7         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |    4944 MB |    4949 MB |   51061 MB |   46117 MB |\n|       from large pool |    4575 MB |    4823 MB |   48966 MB |   44391 MB |\n|       from small pool |     368 MB |     368 MB |    2094 MB |    1726 MB |\n|---------------------------------------------------------------------------|\n| Active memory         |    4944 MB |    4949 MB |   51061 MB |   46117 MB |\n|       from large pool |    4575 MB |    4823 MB |

In [34]:
from pynvml import *

ModuleNotFoundError: No module named 'pynvml'

In [32]:
def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")
print_gpu_utilization()

NameError: name 'nvmlInit' is not defined